# 0. EDA

In [26]:
import pyarrow.parquet as pq
import pandas as pd
import duckdb
import polars as pl

file_path = "./data/parquet/2025-04.parquet"
train_seq_len = 3 

### 1. duckDB: 실제 서비스, 혹은 train pipeline에서 속도를 개선하기 위해선 duckDB가 필요할지도 모름.

In [21]:
con = duckdb.connect()
con.execute(f"""CREATE TABLE data AS SELECT * FROM '{file_path}'""")
# con.execute(f"SELECT * FROM data limit 10").fetchall()

### 2. polars: 가볍게 쓰기엔 얘가 더 좋을 듯. 일단 도즈언

In [ ]:
# read parquet file with polars
df = pl.read_parquet(file_path)

# sort by seq for group by by sequence order
df_sort = df.sort("seq")
df_sort.head(20)

sid,parentSid,device,urlId,url,seq,useless,fullReferStr,sessionId,uTurnPoint,interval,lastOrder,exist,reloadCount,rollback,dateStr,dateHourStr,parentSeq,initialReferId,srcId,termId,campId,contId,medId,gUrlId,prevNodeId,nodePath,exactNodePath,prevCustomId,beuCustomId,nodeCustomPath,beuCustomUserId,beuCustomUserIdForFiltering,loginStatusChange,firstPathname,sampleNumb
str,str,str,i64,str,i64,bool,str,str,bool,f64,bool,bool,i64,bool,str,str,i64,i64,f64,f64,f64,f64,f64,i64,i64,str,str,i64,i64,str,i64,i64,i64,str,i64
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",2,"""https://www.beusable.net/ko/""",0,false,"""Direct""","""07mYi17Q""",false,NaN,true,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,1,NaN,NaN,NaN,NaN,NaN,4930,-1,"""""",null,-1,1,null,0,0,0,"""ko""",80
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",146,"""https://www.beusable.net/ko/ux…",0,false,"""Direct""","""2qVgJwlW""",false,2872.0,false,true,0,false,"""2025-04-01""","""2025-04-01T00""",0,1,NaN,NaN,NaN,NaN,NaN,95149,-1,"""""",null,-1,3,null,0,493,0,"""ko""",97
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",125,"""https://www.beusable.net/signi…",0,false,"""Direct""","""bjzKSe6K""",false,NaN,true,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,1,NaN,NaN,NaN,NaN,NaN,95121,-1,"""""",null,-1,0,null,0,0,0,"""signin""",19
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",2,"""https://www.beusable.net/ko/""",0,false,"""Direct""","""d7hnpV2g""",false,NaN,true,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,1,NaN,NaN,NaN,NaN,NaN,4930,-1,"""""",null,-1,1,null,0,0,0,"""ko""",76
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",1,"""https://www.beusable.net/blog/…",0,false,"""https://www.google.com/""","""h0ugeryN""",false,NaN,true,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,2,NaN,NaN,NaN,NaN,NaN,4989,-1,"""""",null,-1,0,null,0,0,0,"""blog""",62
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",1,"""https://www.beusable.net/blog/…",0,false,"""https://search.naver.com/searc…","""Nc9tVkco""",false,NaN,true,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,2628,NaN,NaN,NaN,NaN,NaN,4989,-1,"""""",null,-1,0,null,0,0,0,"""blog""",19
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",125,"""https://www.beusable.net/signi…",0,false,"""https://analytics.beusable.net…","""nOtPZSMJ""",false,8687.0,false,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,409,NaN,NaN,NaN,NaN,NaN,95121,-1,"""""",null,-1,0,null,0,222,0,"""signin""",79
"""s856b0b83b6""","""s856b0b83b6""","""desktop""",1,"""https://www.beusable.net/blog/…",0,false,"""https://www.google.com/""","""oBScwuWD""",false,NaN,true,false,0,false,"""2025-04-01""","""2025-04-01T00""",0,2,NaN,NaN,NaN,NaN,NaN,4989,-1,"""""",null,-1,0,null,0,0,0,"""blog""",24


In [31]:
# data reform: input + broadcast context
# this form is building model per company

grouped_df = (
    df_sort.group_by("sessionId")
    .agg(
        # input
        pl.col("urlId").alias("urlId"),
        pl.max("seq").alias("seqLength"),
        pl.col("exist").alias("exist"),
        pl.col("rollback").alias("rollback"),
        pl.col("uTurnPoint").alias("uTurnPoint"),
        pl.col("lastOrder").alias("lastOrder"),
        
        # broadcast context
        pl.first("device").alias("device"),
        pl.first("initialReferId").alias("initialReferId")
    )
)


# result = {}
# for row in df_sort.iter_rows(named=True):
#     if result.get(row["sessionId"]):
#         result[row["sessionId"]]["urlId"].append(row["urlId"])
#         result[row["sessionId"]]["exist"].append(row["exist"])
#         result[row["sessionId"]]["rollback"].append(row["rollback"])
#     else:
#         result[row["sessionId"]] = {
#             "urlId": [row["urlId"]], 
#             "exist": [row["exist"]],
#             "rollback": [row["rollback"]],
#             "device": row["device"], 
#             "initialReferId": row["initialReferId"]
#         }

In [32]:
filtered_df = grouped_df.filter(pl.col("seqLength") >= train_seq_len)

In [34]:
final_train_df = filtered_df.sort("seqLength", descending=True)
final_train_df

sessionId,urlId,seqLength,exist,rollback,uTurnPoint,lastOrder,device,initialReferId
str,list[i64],i64,list[bool],list[bool],list[bool],list[bool],str,i64
"""xbcLrJ9A""","[1, 3136, … 3270]",136,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … true]","""desktop""",1
"""QzljjPa8""","[127, 4, … 83770]",86,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … true]","""desktop""",1
"""MXDKTUVP""","[4, 80536, … 80387]",73,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … false]","""desktop""",1
"""CclIacUp""","[4, 83170, … 83183]",68,"[true, true, … true]","[false, false, … false]","[false, false, … true]","[false, false, … false]","""desktop""",3291
"""g4XfQ3xp""","[125, 127, … 7]",62,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … true]","""desktop""",1
…,…,…,…,…,…,…,…,…
"""WL1faEJ5""","[125, 4, … 66265]",3,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … true]","""desktop""",1
"""E9Qha5Hn""","[125, 125, … 48136]",3,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … true]","""desktop""",1
"""RxP5irwB""","[2, 125, … 127]",3,"[true, true, … true]","[false, false, … false]","[false, false, … false]","[false, false, … false]","""desktop""",1


In [37]:
final_train_data = final_train_df.to_dict(as_series=False)
final_train_data


{'sessionId': ['xbcLrJ9A',
  'QzljjPa8',
  'MXDKTUVP',
  'CclIacUp',
  'g4XfQ3xp',
  '3R5NWnQ5',
  'nPg7diub',
  'lLFSyztn',
  'QnTDxzgu',
  'EwN1hLyo',
  'CGxdlMyj',
  'Vw07i3Vn',
  'YePjuvMB',
  'KIaa8kRL',
  'TnARJyUE',
  '1jIb7a0G',
  'ODtilA1O',
  'WgEA7fkn',
  'njFzVhJ3',
  'oloMXDnm',
  'epcKT2se',
  'OWT95nNc',
  'vOhcdwsp',
  'aOWgZjIw',
  'N0WQuLOm',
  'vyRpnW40',
  'QzUHGfSo',
  'D8NH6oaK',
  'IrZYm561',
  'h5758Pxq',
  'jBIr8kfO',
  '32goRfj6',
  '5n0pbFz0',
  '8BV0st6E',
  'j3ydDpVl',
  'PnHd9tOV',
  '3e16jZag',
  'N3kSxSGQ',
  'EZ5Mn6jy',
  '1Qv13oyW',
  'rQTm18zB',
  'PxyoKsF0',
  'ZLRHwwnn',
  '39g9hz6T',
  'wo0N8pFl',
  '2OP5Dk9i',
  'i7wloOcK',
  '4uuNWOeC',
  'aQEwkD7Q',
  'ufEfaFzY',
  'uQZerHnA',
  'mkCtb0ZK',
  'kazAtEYW',
  'DPiA88jc',
  '6O7Cwan1',
  'b3aRBi0S',
  'iD01xXS3',
  'v8uCHDTi',
  'UP3theQy',
  'NXWBCnWR',
  '5lOgTHio',
  'PCOwztLc',
  'HOWQlJr2',
  'oPjsy5Em',
  'gb0N9ajS',
  '455ly5fo',
  'q4lRFWYO',
  'nbu7H19o',
  'lYIhC45n',
  '9EWect2B',
  'wacR

# 1. Model

In [1]:
# model.py
import torch.nn as nn

class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output = nn.Linear(emb_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)         # (B, T, D)
        x = x.permute(1, 0, 2)        # (T, B, D)
        out = self.transformer(x)     # (T, B, D)
        return self.output(out[-1])   # 마지막 위치 출력만 예측

In [ ]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        # x: (B, T, D)
        positions = torch.arange(x.size(1), device=x.device).unsqueeze(0)  # (1, T)
        return x + self.pe(positions)  # broadcasting 됨

class SimpleTransformerWithPositionalEncoding(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, nhead=4, num_layers=2, max_len=512):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.pos_encoder = PositionalEncoding(emb_dim, max_len)

        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output = nn.Linear(emb_dim, vocab_size)

    def generate_causal_mask(self, seq_len, device):
        # 하삼각 causal mask: future 정보 차단
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask.to(device)

    def forward(self, x):
        # x: (B, T)
        x = self.embedding(x)              # (B, T, D)
        x = self.pos_encoder(x)            # (B, T, D)
        x = x.permute(1, 0, 2)             # (T, B, D)

        seq_len = x.size(0)
        mask = self.generate_causal_mask(seq_len, x.device)

        out = self.transformer(x, mask=mask)  # (T, B, D)
        return self.output(out[-1])           # (B, vocab_size) - 마지막 토큰 위치 예측

In [ ]:
# 고정값(장비, 브라우저, 사업군) BroadCast Context Embedding

import torch
import torch.nn as nn

class PositionalEncoding(nn.Module): # 토큰 위치 인코딩. 이거 중요!
    def __init__(self, d_model, max_len=512):
        super().__init__()
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        # x: (B, T, D)
        positions = torch.arange(x.size(1), device=x.device).unsqueeze(0)  # (1, T)
        return x + self.pe(positions)

# set args which are fixed values.
class TransformerWithContext(nn.Module):
    def __init__(
        self,
        vocab_size,
        action_size,
        device_type_size,
        browser_type_size,
        os_type_size,
        emb_dim=128,
        nhead=4,
        num_layers=2,
        max_len=512
    ):
        super().__init__()
        # Embeddings
        self.page_embedding = nn.Embedding(vocab_size, emb_dim)
        self.action_embedding = nn.Embedding(action_size, emb_dim)
        self.device_embedding = nn.Embedding(device_type_size, emb_dim)
        self.browser_embedding = nn.Embedding(browser_type_size, emb_dim)
        self.os_embedding = nn.Embedding(os_type_size, emb_dim)

        self.pos_encoder = PositionalEncoding(emb_dim, max_len)

        # Transformer
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Output layer
        self.output = nn.Linear(emb_dim, vocab_size)

    def generate_causal_mask(self, seq_len, device):
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
        return mask.masked_fill(mask == 1, float('-inf')).to(device)

    def forward(self, page_seq, action_seq, device_id, browser_id, os_id):
        """
        page_seq, action_seq: (B, T)
        device_id, browser_id, os_id: (B,)
        """
        x_page = self.page_embedding(page_seq)     # (B, T, D)
        x_action = self.action_embedding(action_seq)  # (B, T, D)

        # Broadcast context embeddings to (B, T, D)
        device_emb = self.device_embedding(device_id).unsqueeze(1).expand(-1, page_seq.size(1), -1)
        browser_emb = self.browser_embedding(browser_id).unsqueeze(1).expand(-1, page_seq.size(1), -1)
        os_emb = self.os_embedding(os_id).unsqueeze(1).expand(-1, page_seq.size(1), -1)

        # Combine all embeddings
        x = x_page + x_action + device_emb + browser_emb + os_emb
        x = self.pos_encoder(x)       # (B, T, D)
        x = x.permute(1, 0, 2)        # (T, B, D)

        mask = self.generate_causal_mask(x.size(0), x.device)
        x = self.transformer(x, mask=mask)  # (T, B, D)

        return self.output(x[-1])

# 2. Train

In [ ]:
# simple

# train_model.py
import torch
from torch.utils.data import DataLoader, TensorDataset
# from model import SimpleTransformer

# 예시용 dummy 데이터
vocab_size = 100
X = torch.randint(0, vocab_size, (500, 10))  # (samples, seq_len)
y = torch.randint(0, vocab_size, (500,))     # (samples,)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SimpleTransformer(vocab_size).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

torch.save(model.state_dict(), "model.pt")

In [ ]:
# train_model.py
import torch
from torch.utils.data import DataLoader, TensorDataset
# from model import SimpleTransformerWithPositionalEncoding

# 설정
vocab_size = 100
seq_len = 10
batch_size = 32
epochs = 10
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# 예제 데이터 (임시)
X = torch.randint(0, vocab_size, (500, seq_len))  # (samples, seq_len)
y = torch.randint(0, vocab_size, (500,))          # (samples,)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 정의
model = SimpleTransformerWithPositionalEncoding(vocab_size=vocab_size).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 훈련 루프
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"📈 Epoch {epoch+1}: Loss = {total_loss:.4f}")

# 모델 저장
torch.save(model.state_dict(), "model.pt")
print("✅ 모델이 저장되었습니다: model.pt")

In [ ]:
# train_model_with_context.py
import torch
from torch.utils.data import DataLoader, TensorDataset

# 설정
vocab_size = 100 # 토큰 사이즈
action_size = 5 # 고정값 갯수: click, scroll, hover, focus, submit
device_size = 10 # 고정값 갯수: pc, mobile, tablet
browser_size = 10 # 고정값 갯수: chrome, firefox, safari, edge, ie, opera, other
os_size = 10 # 고정값 갯수: windows, mac, linux, ios, android, other
seq_len = 10 # 토큰 갯수이자, 시퀀스 길이: 
batch_size = 32 
epochs = 10
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# 1️⃣ 샘플 데이터 생성
num_samples = 500
page_seq = torch.randint(0, vocab_size, (num_samples, seq_len))
action_seq = torch.randint(0, action_size, (num_samples, seq_len))
device_ids = torch.randint(0, device_size, (num_samples,))
browser_ids = torch.randint(0, browser_size, (num_samples,))
os_ids = torch.randint(0, os_size, (num_samples,))
labels = torch.randint(0, vocab_size, (num_samples,))  # 다음 페이지 예측용

# 2️⃣ Dataset 구성
dataset = TensorDataset(page_seq, action_seq, device_ids, browser_ids, os_ids, labels)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 3️⃣ 모델 초기화
model = TransformerWithContext(
    vocab_size=vocab_size,
    action_size=action_size,
    device_type_size=device_size,
    browser_type_size=browser_size,
    os_type_size=os_size,
).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 4️⃣ 학습 루프
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in loader:
        page_seq, action_seq, device_id, browser_id, os_id, label = [x.to(device) for x in batch]

        optimizer.zero_grad()
        logits = model(page_seq, action_seq, device_id, browser_id, os_id)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"📈 Epoch {epoch+1}: Loss = {total_loss:.4f}")

# 5️⃣ 모델 저장
torch.save(model.state_dict(), "context_model.pt")
print("✅ 모델이 저장되었습니다: context_model.pt")